In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist

In [2]:
# carregar o dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
# calcular o quantas classes existem
num_labels = len(np.unique(y_train))

# transformação das classes em valores binários
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# definindo a dimensao da imagem
image_size = x_train.shape[1]

In [4]:
print(x_train.shape)

(60000, 28, 28)


In [5]:
# redimensionamento do vetor de entradas
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])

# normalização das entradas
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [6]:
# definindo o tamanho da entrada como sendo o tamanho da imagem com um canal
input_shape = (image_size, image_size, 1)

# tamanho do batch que será usado no treinamento
batch_size = 128

# tamanho da janela de convolução
kernel_size = 3


pool_size = 2
filters = 64

# dropout rate
dropout = 0.2

In [7]:
# iniciando a criação da rede neural
model = Sequential()

# primeira camada convolucional
# possui 64 filtros e kernel_size 3x3
# já embutida a função de ativação 'relu'
# o tamanho da entrada é especificada como sendo o tamanho da imagem
model.add(Conv2D(filters=filters,
                 kernel_size=kernel_size,
                 activation='relu',
                 input_shape=input_shape))

# pooling de 3, fazendo com que o tamanho vá para (13,13,64)                 
model.add(MaxPooling2D(pool_size))

# adicionando mais uma convolução ficando com tamanho (11,11,64)
model.add(Conv2D(filters=filters,
                 kernel_size=kernel_size,
                 activation='relu'))

# mais uma camada de pooling, com o tamnho indo para (5,5,64)
model.add(MaxPooling2D(pool_size))

# mais uma camada de convolução com dimensao (3,3,64)
model.add(Conv2D(filters=filters,
                 kernel_size=kernel_size,
                 activation='relu'))

# muda o formato para 3x3x64 = 576
model.add(Flatten())

# dropout added as regularizer
# camada dropout adicionada como regularizer, aleatoriamente joga inputs pra 0 com uma certa frequencia, evitando overfitting
# não há mudança de tamanho
model.add(Dropout(dropout))

# camada de saída que compoe um vetor de uns e zeros
model.add(Dense(num_labels))

# ativação
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [8]:
# otimizador utilizado é o Adam
# função loss categorical_crossentropy utilizado, pois há mais de dois labels
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

              
# treinamento da rede com 10 epocas
model.fit(x_train, y_train, epochs=10, batch_size=batch_size)

Epoch 1/10
469/469 [==============================] - 36s 75ms/step - loss: 0.2708 - accuracy: 0.9174
Epoch 2/10
469/469 [==============================] - 36s 76ms/step - loss: 0.0707 - accuracy: 0.9779
Epoch 3/10
469/469 [==============================] - 40s 84ms/step - loss: 0.0497 - accuracy: 0.9846
Epoch 4/10
469/469 [==============================] - 38s 82ms/step - loss: 0.0403 - accuracy: 0.9878
Epoch 5/10
469/469 [==============================] - 46s 98ms/step - loss: 0.0341 - accuracy: 0.9899
Epoch 6/10
469/469 [==============================] - 40s 85ms/step - loss: 0.0285 - accuracy: 0.9913
Epoch 7/10
469/469 [==============================] - 40s 85ms/step - loss: 0.0248 - accuracy: 0.9922
Epoch 8/10
469/469 [==============================] - 43s 91ms/step - loss: 0.0223 - accuracy: 0.9931
Epoch 9/10
469/469 [==============================] - 39s 83ms/step - loss: 0.0193 - accuracy: 0.9936
Epoch 10/10
469/469 [==============================] - 39s 82ms/step - loss: 0.016

In [37]:
_, acc = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))


Test accuracy: 99.3%


In [25]:
train_results = model.predict(x_train)
test_results = model.predict(x_test)

313/313 [==============================] - 2s 6ms/step


In [35]:
total_train = len(train_results)
correct_train = 0
for i,pred in enumerate(train_results):
    classification = np.round(pred,0)
    if (classification == y_train[i]).all:
        correct_train += 1

tx_acerto_train = correct_train/total_train
print(f'Acertos treino em porcentagem: {tx_acerto_train*100}%')

Acertos treino em porcentagem: 100.0%


In [36]:
total_test = len(test_results)
correct_test = 0
for i,pred in enumerate(test_results):
    classification = np.round(pred,0)
    if (classification == y_test[i]).all:
        correct_test += 1

tx_acerto_test = correct_test/total_test
print(f'Acertos teste em porcentagem: {tx_acerto_test*100}%')

Acertos teste em porcentagem: 100.0%
